In [ ]:
# import pandas as pd
# prices = pd.read_csv('data/prices_hist.csv')

In [ ]:
# # Данные по ценам арматуры
# prices.head()

# Importing packages

In [1]:
!pip install -r requirements.txt

  Using cached holidays-0.19-py3-none-any.whl (208 kB)
  Using cached nfoursid-1.0.1-py3-none-any.whl (16 kB)
  Using cached pmdarima-2.0.2-cp39-cp39-win_amd64.whl (572 kB)
  Using cached prophet-1.1.2-py3-none-win_amd64.whl (12.1 MB)
  Using cached pyod-1.0.7-py3-none-any.whl
  Using cached statsforecast-1.4.0-py3-none-any.whl (91 kB)
  Using cached tbats-1.1.2-py3-none-any.whl (43 kB)
  Using cached xgboost-1.7.3-py3-none-win_amd64.whl (89.1 MB)
  Using cached black-22.3.0-cp39-cp39-win_amd64.whl (1.1 MB)
  Using cached flake8-4.0.1-py2.py3-none-any.whl (64 kB)
  Using cached isort-5.10.1-py3-none-any.whl (103 kB)
  Using cached pre_commit-3.0.4-py2.py3-none-any.whl (202 kB)
  Using cached pytest_cov-4.0.0-py3-none-any.whl (21 kB)
  Using cached pyupgrade-2.31.0-py2.py3-none-any.whl (54 kB)
  Using cached testfixtures-7.1.0-py3-none-any.whl (102 kB)
  Using cached bump2version-1.0.1-py2.py3-none-any.whl (22 kB)
  Using cached ipython-8.8.0-py3-none-any.whl (775 kB)
  Using cached ipy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires libclang>=13.0.0, which is not installed.
tensorflow 2.9.1 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
torchvision 0.10.0+cpu requires torch==1.9.0, but you have torch 1.13.1 which is incompatible.
torchaudio 0.9.0 requires torch==1.9.0, but you have torch 1.13.1 which is incompatible.
tensorflow 2.9.1 requires flatbuffers<2,>=1.12, but you have flatbuffers 2.0 which is incompatible.
tensorflow 2.9.1 requires gast<=0.4.0,>=0.2.1, but you have gast 0.5.3 which is incompatible.
tensorflow 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.



  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached statsmodels-0.13.5-cp39-cp39-win_amd64.whl (9.2 MB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
  Using cached cmdstanpy-1.1.0-py3-none-any.whl (83 kB)
  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached cfgv-3.3.1-py2.py3-none-any.whl (7.3 kB)
  Using cached virtualenv-20.19.0-py3-none-any.whl (8.7 MB)
  Using cached identify-2.5.17-py2.py3-none-any.whl (98 kB)
  Using cached nodeenv-1.7.0-py2.py3-none-any.whl (21 kB)
  Using cached coverage-7.1.0-cp39-cp39-win_amd64.whl (202 kB)
  Using cached torchmetrics-0.11.1-py3-none-any.whl (517 kB)
  Using cached torch-1.13.1-cp39-cp39-win_amd64.whl (162.5 MB)
  Using cached lightning_utilities-0.6.0.post0-py3-none-any.whl (18 kB)
  Using cached ephem-4.1.4-cp39-cp39-win_amd64.whl (1.4 MB)
  Using cached numpy-1.21.6-cp39-cp39-win_amd64.whl (14.0 MB)
  Using cached distlib-0.3.6-py2.py3-none-any.whl (468 kB)
  Using cached tex

In [ ]:
!pip install darts

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from darts.timeseries import TimeSeries
from darts.models import {
    LinearRegressionModel,
    ARIMA,
    RandomForest,
    Theta,
    RegressionEnsembleModel,
    NBEATSModel
}
from darts.metrics import mape, nse

import warnings
warnings.filterwarnings('ignore')

# Reproducibility

In [ ]:
import torch
torch.manual_seed(0)

import random
random.seed(0)
import numpy as np
np.random.seed(0)